# Callbacks version 1

In [ ]:
%install '.package(path: "$cwd/FastaiNotebooks")' FastaiNotebooks

Installing packages:
	.package(path: "/home/ubuntu/fastai_docs/dev_swift/FastaiNotebooks")
		FastaiNotebooks
With SwiftPM flags: []
Working in: /tmp/tmp6r43l7fy
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 1.16s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'FastaiNotebooks' (5 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Loading library...
Installation complete!


## Load data

In [ ]:
import FastaiNotebooks

In [ ]:
// export
import Path
import TensorFlow

In [ ]:
var (xTrain,yTrain,xValid,yValid) = loadMNIST(path: mnistPath, flat: true)

In [ ]:
let (n,m) = (Int(xTrain.shape[0]),Int(xTrain.shape[1]))
let c = yTrain.max()+1
print(n,m,c)

60000 784 10


Those can't be used to define a model cause they're not Ints though...

In [ ]:
let (n,m) = (60000,784)
let c = 10
let nHid = 50

In [ ]:
// export
public struct BasicModel: Layer {
    public var layer1: Dense<Float>
    public var layer2: Dense<Float>
    
    public init(nIn: Int, nHid: Int, nOut: Int){
        layer1 = Dense(inputSize: nIn, outputSize: nHid, activation: relu)
        layer2 = Dense(inputSize: nHid, outputSize: nOut)
    }
    
    @differentiable
    public func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return input.sequenced(in: context, through: layer1, layer2)
    }
}

In [ ]:
var model = BasicModel(nIn: m, nHid: nHid, nOut: c)

In [ ]:
// export
public struct DataBunch<Element> where Element: TensorGroup{
    public var train: Dataset<Element>
    public var valid: Dataset<Element>
    
    public init(train: Dataset<Element>, valid: Dataset<Element>) {
        self.train = train
        self.valid = valid
    }
}

In [ ]:
//export
public func mnistDataBunch(path: Path = mnistPath, flat: Bool = false, bs: Int = 64
                          ) -> DataBunch<DataBatch<Tensor<Float>, Tensor<Int32>>>{
    let (xTrain,yTrain,xValid,yValid) = loadMNIST(path: path, flat: flat)
    return DataBunch(train: Dataset(elements:DataBatch(xb:xTrain, yb:yTrain)).batched(Int64(bs)), 
                     valid: Dataset(elements:DataBatch(xb:xValid, yb:yValid)).batched(Int64(bs)))
}

In [ ]:
let data = mnistDataBunch(flat: true)

## Learner (Marc's version)

In [ ]:
// Please pick a better name for me! :-)
enum CallbackException {
    case cancelTraining
    case cancelEpoch
    case cancelBatch
}

enum CallbackEvent {
    // I haven't implemented all the events.
    case beginFit
    case beginEpoch
    case beginBatch
    case beginValidate
    case afterForwardsBackwards
    case afterFit
}

func defaultCallback(e: CallbackEvent) {}

Basic class

In [ ]:
class Learner<Opt: Optimizer, Labels: TensorGroup>
where Opt.Model.CotangentVector == Opt.Model.AllDifferentiableVariables,
      Opt.Model.Input: TensorGroup
{
    typealias Model = Opt.Model
    var model: Model
    
    typealias Inputs = Model.Input
    // I'm getting some crashes in AD-generated code if I put a `lossFunc` in the learner.
    // So I'm putting a `lossWithGradient` for now, to work around this.
    // (model, context, inputs, labels) -> (loss, grad)
    typealias LossWithGradient = (Model, Context, Inputs, Labels) -> (Tensor<Float>, Model.AllDifferentiableVariables)
    var lossWithGradient: LossWithGradient
    
    var optimizer: Opt
    
    typealias Data = DataBunch<DataBatch<Inputs, Labels>>
    var data: Data
    
    var context: Context = Context(learningPhase: .training)

    typealias Callback = (CallbackEvent) throws -> ()    
    var callback: Callback = defaultCallback
    
    var loss: Tensor<Float> = Tensor(0)
    var grad: Model.AllDifferentiableVariables = Model.AllDifferentiableVariables.zero
    
    var inTrain: Bool = false
    var epoch: Int = 0
    var epochs: Int = 0
    var nEpochs: Float = 0.0
    var nIter: Int = 0
    var iters: Int = 0
    
    init(
        model: Model,
        lossWithGradient: @escaping LossWithGradient,
        optimizer: Opt,
        data: Data
    ) {
        self.model = model
        self.lossWithGradient = lossWithGradient
        self.optimizer = optimizer
        self.data = data
    }
}

Then let's write the parts of the training loop:

In [ ]:
extension Learner{
    func trainOneBatch(xb: Inputs, yb: Labels) throws {
        try callback(.beginBatch)
        (self.loss, self.grad) = lossWithGradient(model, self.context, xb, yb)
        defer {
            // Zero out the loss & gradient to ensure stale values aren't used.
            self.loss = Tensor(0)
            self.grad = Model.AllDifferentiableVariables.zero        
        }
        try callback(.afterForwardsBackwards)
        if self.inTrain {optimizer.update(&model.allDifferentiableVariables, along: self.grad)}
    }
    
    func trainOneEpoch() throws {
        //self.iters = length of the current dataset
        for batch in self.inTrain ? self.data.train : self.data.valid {
            do { try trainOneBatch(xb: batch.xb, yb: batch.yb)} 
            catch CallbackException.cancelBatch {}  // Continue
        }
    }
}

And the whole fit function.

In [ ]:
extension Learner{
    func fit(epochs: Int) throws {
        self.epochs = epochs
        do {
            try callback(.beginFit)
            defer {
                do { try callback(.afterFit) } 
                catch { print("Error during callback(.afterFit): \(error)")}
            }
            for epoch in 1...epochs {
                self.epoch = epoch
                try callback(.beginEpoch)
                do { try trainOneEpoch()} 
                catch let error as CallbackException where error != .cancelTraining {}  // Continue
                try callback(.beginValidate)
                do { try trainOneEpoch()} 
                catch let error as CallbackException where error != .cancelTraining {}  // Continue
            }
        } catch is CallbackException {}  // Catch all CallbackExceptions.
    }
}

In [ ]:
// Helper so you don't need to do the chaining yourself. :-)
func chainCallback<Opt, Labels>(on learner: Learner<Opt, Labels>, newCallback: @escaping (CallbackEvent) throws -> ()) {
    let existingCallback = learner.callback
    learner.callback = { event in
        try newCallback(event)
        try existingCallback(event)
    }
}

In [ ]:
func installTrainEval<Opt, Labels>(on learner: Learner<Opt, Labels>) {
    chainCallback(on: learner) { event in
        switch event {
        case .beginFit:
            learner.nEpochs = 0.0
            learner.nIter = 0
        case .beginEpoch:
            learner.nEpochs = Float(learner.epoch)
            learner.context = Context(learningPhase: .training)
            learner.inTrain = true
            print("Starting new epoch: \(learner.epoch) of \(learner.epochs)!")
        case .afterForwardsBackwards:
            if learner.inTrain{
                //learn.nEpochs += 1./self.iters Uncomment when length of datasets added
                learner.nIter   += 1
            }
        case .beginValidate:
            learner.context = Context(learningPhase: .inference)
            learner.inTrain = false
        default: break
        }
    }
}

### Test

In [ ]:
func lossWithGrad(
    model: BasicModel,
    in context: Context,
    inputs: Tensor<Float>,
    labels: Tensor<Int32>
) -> (Tensor<Float>, BasicModel.AllDifferentiableVariables) {
    return model.valueWithGradient { model -> Tensor<Float> in
        let predictions = model.applied(to: inputs, in: context)
        return softmaxCrossEntropy(logits: predictions, labels: labels)
    }
}

In [ ]:
let opt = SGD<BasicModel, Float>(learningRate: 1e-2)

In [ ]:
let learner = Learner(
    model: model,
    lossWithGradient: lossWithGrad,
    optimizer: opt,
    data: data)

In [ ]:
installTrainEval(on: learner)

In [ ]:
learner.fit(epochs: 2)

Starting new epoch: 1 of 2!
Starting new epoch: 2 of 2!
